#**Imports y Cargado del dataset**

In [0]:
%tensorflow_version 1.x
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.layers import GaussianNoise as GN
from keras.optimizers import SGD

from keras.callbacks import LearningRateScheduler as LRS
from keras.preprocessing.image import ImageDataGenerator


batch_size = 100
num_classes = 10
epochs = 75


#### LOAD AND TRANSFORM
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

print(x_train.shape)
print(x_test.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

TensorFlow 1.x selected.


Using TensorFlow backend.


170500096/170498071 [==============================] - 4s 0us/step
(50000, 32, 32, 3)
(10000, 32, 32, 3)


# **Data Augmentation**

In [0]:
## DEFINE A DATA AUGMENTATION GENERATOR

datagen = ImageDataGenerator(
    width_shift_range=0.3,
    height_shift_range=0.3,
    rotation_range=20,
    zoom_range=[1.0,1.2],
    horizontal_flip=True)

print("Data Augmentation created")

Data Augmentation created


# **Convolutional Block**

In [0]:
## DEF A BLOCK CONV + BN + GN + MAXPOOL
def CBGN(model,filters,ishape=0, num_blocks=1, C=False):
  for i in range(num_blocks):
    if (ishape!=0):
      model.add(Conv2D(filters, (3, 3), padding='same',
                  input_shape=ishape))
    else:
      if i >= 2 and C:
        model.add(Conv2D(filters, (1, 1), padding='same'))
      else:
        model.add(Conv2D(filters, (3, 3), padding='same'))

    model.add(BN())
    model.add(GN(0.3))
    model.add(Activation('relu'))

  model.add(MaxPooling2D(pool_size=(2, 2)))
  
  
  return model

print("Convolutional block defined")

Convolutional block defined


# **Topología**

In [0]:
## DEF VGG TOPOLOGY  
def def_vgg(num_blocks64=1,num_blocks128=1,num_blocks256=2,num_blocks512a=2,num_blocks512b=2, C=False):
    vgg = Sequential()

    vgg=CBGN(vgg,64,x_train.shape[1:],num_blocks64)
    vgg=CBGN(vgg,128,0,num_blocks128)
    vgg=CBGN(vgg,256,0,num_blocks256,C)
    vgg=CBGN(vgg,512,0,num_blocks512a,C)
    vgg=CBGN(vgg,512,0,num_blocks512b,C)

    vgg.add(Flatten())
    vgg.add(Dense(4096))
    vgg.add(Activation('relu'))
    vgg.add(Dense(4096))
    vgg.add(Activation('relu'))

    vgg.add(Dense(num_classes))
    vgg.add(Activation('softmax'))

    return vgg

# **Definición del modelo y summary**

In [0]:
#model = def_vgg(1,1,2,2,2)#A

#model = def_vgg(2,2,2,2,2)#B

#model = def_vgg(2,2,3,3,3,True)#C

#model = def_vgg(2,2,3,3,3)#D

model = def_vgg(2,2,4,4,4)#E

model.summary()

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_194 (Conv2D)          (None, 32, 32, 64)        1792      
_________________________________________________________________
batch_normalization_194 (Bat (None, 32, 32, 64)        256       
_________________________________________________________________
gaussian_noise_194 (Gaussian (None, 32, 32, 64)        0         
_________________________________________________________________
activation_251 (Activation)  (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_195 (Conv2D)          (None, 32, 32, 64)        36928     
_________________________________________________________________
batch_normalization_195 (Bat (None, 32, 32, 64)        256       
_________________________________________________________________
gaussian_noise_195 (Gaussian (None, 32, 32, 64)      

# **Optimizador y compilación**

In [0]:
## OPTIM AND COMPILE
opt = SGD(lr=0.1, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# **Learning Rate**

In [0]:
# DEFINE A LEARNING RATE SCHEDULER
def scheduler(epoch):
    if epoch < 25:
        return .1
    elif epoch < 50:
        return 0.01
    else:
        return 0.001

set_lr = LRS(scheduler)

print("Learning Rate scheduled")

Learning Rate scheduled


# **Training**

In [0]:
## TRAINING with DA and LRA
history=model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),
                            steps_per_epoch=len(x_train) / batch_size, 
                            epochs=epochs,
                            validation_data=(x_test, y_test),
                            callbacks=[set_lr],
                            verbose=1)

Epoch 1/75
500/500 [==============================] - 40s 80ms/step - loss: 1.8908 - accuracy: 0.2874 - val_loss: 2.0525 - val_accuracy: 0.3078
Epoch 2/75
500/500 [==============================] - 35s 70ms/step - loss: 1.4776 - accuracy: 0.4561 - val_loss: 1.6458 - val_accuracy: 0.4666
Epoch 3/75
500/500 [==============================] - 35s 71ms/step - loss: 1.2453 - accuracy: 0.5512 - val_loss: 1.4526 - val_accuracy: 0.5561
Epoch 4/75
500/500 [==============================] - 35s 70ms/step - loss: 1.0886 - accuracy: 0.6124 - val_loss: 1.2521 - val_accuracy: 0.6196
Epoch 5/75
500/500 [==============================] - 35s 71ms/step - loss: 0.9878 - accuracy: 0.6540 - val_loss: 1.0364 - val_accuracy: 0.6625
Epoch 6/75
500/500 [==============================] - 35s 71ms/step - loss: 0.9014 - accuracy: 0.6873 - val_loss: 1.0797 - val_accuracy: 0.6696
Epoch 7/75
500/500 [==============================] - 35s 71ms/step - loss: 0.8358 - accuracy: 0.7118 - val_loss: 1.0737 - val_accuracy:

# **Test**

In [0]:
## TEST
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

10000/10000 [==============================] - 3s 274us/step
Test loss: 0.3182380014955997
Test accuracy: 0.8978000283241272
